# Подготовка модели распознавания рукописных букв и цифр

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import ToTensor, Compose, Normalize
from torchinfo import summary
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [2]:
from torchvision.datasets import EMNIST
dataset = EMNIST('data/', 'balanced', download=False)

In [ ]:
import os
import pickle
labels_path = os.path.join('myapp', 'labelz.pickle')
with open(labels_path, 'rb') as f:
    labelz = pickle.load(f)

In [41]:
train_dataset = EMNIST('data/', 'balanced', train=True, download=False)
val_dataset = EMNIST('data/', 'balanced',train=False)

print('Train:', len(train_dataset))
print('Valid:', len(val_dataset))

for i in range(50):
    img, lbl = train_dataset[i]
    print(labelz[lbl], img.size)
    display(img)

Train: 112800
Valid: 18800
r (28, 28)


a (28, 28)


n (28, 28)


F (28, 28)


4 (28, 28)


...

9 (28, 28)


t (28, 28)


J (28, 28)


U (28, 28)


d (28, 28)


In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

train_dataset = EMNIST('data/', 'balanced', train=True, download=False, transform=transform)
val_dataset = EMNIST('data/', 'balanced', train=False, transform=transform)



In [5]:
train_dataset_l, train_dataset_s = random_split(train_dataset, [int(len(train_dataset)*0.8), int(len(train_dataset)*0.2)])  

In [15]:
train_loader = DataLoader(train_dataset, batch_size=1000, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1000)

In [32]:
class CNN_for_EMNIST(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=1,
                      out_channels=32,
                      kernel_size=3,
                      padding=1),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(in_channels=32,
                      out_channels=64,
                      kernel_size=3,
                      padding=1),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            

            nn.Flatten(),

            nn.LazyLinear(256),
            nn.LeakyReLU(),
            nn.Dropout(0.3),
            nn.LazyLinear(128),
            nn.LeakyReLU(),
            
            nn.Linear(in_features=128, out_features=47)
            
        )

    def forward(self,x):
        return self.model(x)

In [16]:
def train(model, optimizer, loss_f, train_loader, val_loader, n_epoch, val_fre):
    model.train()
    for epoch in range(n_epoch):
        loss_sum = 0
        print(f'Epoch: {epoch}')
        for step, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data).squeeze(1)
            loss = loss_f(output, target)
            loss.backward()
            optimizer.step()

            loss_sum += loss.item()

            if step % 10 == 0:
                print(f'Iter: {step} \tLoss: {loss.item()}')

        print(f'Mean Train Loss: {loss_sum / (step + 1):.6f}', end='\n\n')

        if epoch % val_fre == 0:
            validate(model, val_loader)

def validate(model, val_loader):
    model.eval()
    loss_sum = 0
    correct = 0
    for step, (data, target) in enumerate(val_loader):
        with torch.no_grad():
            output = model(data).squeeze(1)
            loss = loss_f(output, target)
        loss_sum += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    acc = correct / len(val_loader.dataset)
    print(f'Val Loss: {loss_sum / (step + 1):.6f} \tAccuracy: {acc}')
    model.train()

In [33]:
model = CNN_for_EMNIST()
loss_f = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

n_epoch = 30
val_fre = 2

train(model, optimizer, loss_f, train_loader, val_loader, n_epoch, val_fre)
# torch.save(model.state_dict(), 'checkpoints/cnn.pth')
#model.load_state_dict(torch.load('checkpoints/cnn.pth'))
validate(model, val_loader)

Epoch: 0
Iter: 0 	Loss: 3.8484671115875244
Iter: 10 	Loss: 3.840355634689331
Iter: 20 	Loss: 3.6324594020843506
Iter: 30 	Loss: 3.0425806045532227
Iter: 40 	Loss: 2.4567606449127197
Iter: 50 	Loss: 1.9130405187606812
Iter: 60 	Loss: 1.536525845527649
Iter: 70 	Loss: 1.217049241065979
Iter: 80 	Loss: 1.1117042303085327
Iter: 90 	Loss: 1.0022014379501343
Iter: 100 	Loss: 0.9017571806907654
Iter: 110 	Loss: 0.8680554628372192
Mean Train Loss: 2.087238

Val Loss: 0.650398 	Accuracy: 0.7868085106382978
Epoch: 1
Iter: 0 	Loss: 0.7895172238349915
Iter: 10 	Loss: 0.7382951378822327
Iter: 20 	Loss: 0.7310895919799805
Iter: 30 	Loss: 0.6956156492233276
Iter: 40 	Loss: 0.6157835721969604
Iter: 50 	Loss: 0.6983235478401184
Iter: 60 	Loss: 0.5906784534454346
Iter: 70 	Loss: 0.6700958013534546
Iter: 80 	Loss: 0.5949503183364868
Iter: 90 	Loss: 0.5908921957015991
Iter: 100 	Loss: 0.6059530377388
Iter: 110 	Loss: 0.5071703791618347
Mean Train Loss: 0.640854

Epoch: 2
Iter: 0 	Loss: 0.5580134987831116


In [36]:
torch.save(model.state_dict(), 'model_exp/mod1608_087_no_aug_no_batch.pth')

In [35]:
n_epoch = 5
val_fre = 2

train(model, optimizer, loss_f, train_loader, val_loader, n_epoch, val_fre)
# torch.save(model.state_dict(), 'checkpoints/cnn.pth')
#model.load_state_dict(torch.load('checkpoints/cnn.pth'))
validate(model, val_loader)

Epoch: 0
Iter: 0 	Loss: 0.2541680634021759
Iter: 10 	Loss: 0.2634616792201996
Iter: 20 	Loss: 0.2749045193195343
Iter: 30 	Loss: 0.2627241015434265
Iter: 40 	Loss: 0.25732171535491943
Iter: 50 	Loss: 0.23439337313175201
Iter: 60 	Loss: 0.24564145505428314
Iter: 70 	Loss: 0.2396508902311325
Iter: 80 	Loss: 0.25704628229141235
Iter: 90 	Loss: 0.2783668339252472
Iter: 100 	Loss: 0.2375781238079071
Iter: 110 	Loss: 0.24274341762065887
Mean Train Loss: 0.260864

Val Loss: 0.454082 	Accuracy: 0.8653191489361702
Epoch: 1
Iter: 0 	Loss: 0.21537184715270996
Iter: 10 	Loss: 0.24320003390312195
Iter: 20 	Loss: 0.2712230086326599
Iter: 30 	Loss: 0.20857346057891846
Iter: 40 	Loss: 0.2777753174304962
Iter: 50 	Loss: 0.24302434921264648
Iter: 60 	Loss: 0.22818797826766968
Iter: 70 	Loss: 0.23688969016075134
Iter: 80 	Loss: 0.29370206594467163
Iter: 90 	Loss: 0.2768579423427582
Iter: 100 	Loss: 0.3062811493873596
Iter: 110 	Loss: 0.2907228469848633
Mean Train Loss: 0.252310

Epoch: 2
Iter: 0 	Loss: 0